# Data Extraction
We had to first collect a decent number of random wallets from Etherscan and then retrieve the necessary predictors.

## Attempt 1: Web Scraper

We first attempted to use a website scraper to retrieve data from Etherscan. We learnt how to write a website scraper and implemented it for our use. 

In [ ]:
# In command line (example:)
# C:\Users\your name\AppData\Local\Programs\Python\Python39\Scripts>
# pip install requests
# pip install beautifulsoup4

import csv
import sys
import datetime
import requests
from time import sleep
from bs4 import BeautifulSoup

def scraper(num_pages=4, req_delay=0.1):
  timestamp = datetime.datetime.now().strftime ("%Y%m%d_%H%M%S")
  
  print("%d pages to parse with delay of %f seconds between each page" % (num_pages, req_delay))
  api_url = "https://etherscan.io/contractsVerified/"
  
  with open('ExchangeAccounts-'+timestamp+'.csv', 'w') as csvfile:
    fieldnames = ['addr', 'name_tag', 'balance', 'tx_count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for i in range(0, num_pages):
      url = api_url + str('?subcatid=undefined&size=100&start=') + str(i*100) + str('&col=1&order=asc')
      sleep(req_delay)
      response = requests.get(url)
      print("URL: %s, Status: %s" % (url, response.status_code))

      content = response.content
      soup = BeautifulSoup(content, 'html.parser')

      for row in soup.select('table.table-hover tbody tr'):
        cells = row.findAll('td')
        cells = map(lambda x: x.text, cells)
        addr, name_tag, balance, tx_count = cells
        writer.writerow({
          'addr': addr,
          'name_tag': name_tag,
          'balance': balance,
          'tx_count': tx_count,
        })

def main():
  if len(sys.argv) > 2:
    scraper(int(sys.argv[1]), float(sys.argv[2]))
  elif len(sys.argv) == 2:
    scraper(int(sys.argv[1]))
  else:
    scraper()

if __name__ == "__main__":
  main()

This scraper we wrote works for non-password protected websites. However, some websites we are getting our data from are password protected, and would require more work on our scraper, which we have just learnt and are not as familiar with. 

Thus, we changed our method of retrieving data to retrieving through APIs. 

## Attempt 2: APIs
We went to Etherscan, and collected a bunch of wallets that were tagged to CEXs in Etherscan, and other random user wallets. We then called some APIs, to retrieve predictors that we thought might be relevant, like Ether Balance, Transaction Count etc.

In [8]:
import numpy as np
import pandas as pd
import requests

In [1]:
API_KEY = "H7Y6I5GQ2RN5PJ8VSAYFNEX2HQDGMPWWIZ"

In [30]:
wallets = pd.read_csv('wallets.csv')
walletDF = pd.DataFrame(wallets)
walletDF

,Unnamed: 0,WID,Label,Eth Balance,Txn Count,Exchange,ERC 20 Txn,ERC 20 Txn In,ERC 20 Txn Out,ERC721 Txn Out,ERC721 Txn In,ERC721 Txn
0,0,0x4dc98c79a52968a6c20ce9a7a08d5e8d1c2d5605,NaN,0.00477614 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xbb3fd383d1c5540e52ef0a7bcb9433375793aeaf,NaN,5.25604866 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0x1ccbdff6336b1027995a27a77b41fa87eb6608a3,NaN,0.06097497 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0x05f51aab068caa6ab7eeb672f88c180f67f17ec7,ABCC,0 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x2ddd202174a72514ed522e77972b461b03155525,Alcumex Exchange,0 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
839,839,0xfba17aca0797f72bd8152c5e519100c0c4caf0ad,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
840,840,0xfbbe05954c6b138999548171c272a1b109d89471,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
841,841,0xfefa8d5093ffd962a094fdba722a2c30d1c7a822,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
842,842,0xff64a8933e05c9d585ab72db95d207ebee9be5a8,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


The Ether balance for the non CEX wallets are all empty, so we have to call one of Etherscan's API to populate it.
We are using a timeout to implement a delay every 5 API calls, so we do not exceed our free limit.

In [31]:
import time
count = 0
for index, row in walletDF.iterrows():
    if pd.isna(row['Eth Balance']):
        count += 1
        baseURL = "https://api.etherscan.io/api?module=account&action=balance&address=" + row["WID"] + "&tag=latest&apikey=" + API_KEY
        response = requests.get(baseURL)
        balance = int (response.json()['result']) / 10**18
        walletDF.loc[index, 'Eth Balance'] = balance

        if count == 5:
            time.sleep(2)
            count = 0

walletDF

,Unnamed: 0,WID,Label,Eth Balance,Txn Count,Exchange,ERC 20 Txn,ERC 20 Txn In,ERC 20 Txn Out,ERC721 Txn Out,ERC721 Txn In,ERC721 Txn
0,0,0x4dc98c79a52968a6c20ce9a7a08d5e8d1c2d5605,NaN,0.00477614 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xbb3fd383d1c5540e52ef0a7bcb9433375793aeaf,NaN,5.25604866 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0x1ccbdff6336b1027995a27a77b41fa87eb6608a3,NaN,0.06097497 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0x05f51aab068caa6ab7eeb672f88c180f67f17ec7,ABCC,0 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x2ddd202174a72514ed522e77972b461b03155525,Alcumex Exchange,0 Ether,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
839,839,0xfba17aca0797f72bd8152c5e519100c0c4caf0ad,NaN,0.031192,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
840,840,0xfbbe05954c6b138999548171c272a1b109d89471,NaN,0.001795,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
841,841,0xfefa8d5093ffd962a094fdba722a2c30d1c7a822,NaN,0.035719,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
842,842,0xff64a8933e05c9d585ab72db95d207ebee9be5a8,NaN,30106.172274,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


Now, we need to get the Transaction Count for all our wallets.

In [33]:
for index, row in walletDF.iterrows():
    if pd.isna(row['Txn Count']):
        
        wallet = row['WID']
        baseURL = "https://api.etherscan.io/api?module=account&action=txlist&address=" + wallet + "&startblock=0&endblock=99999999&sort=asc&apikey=" + API_KEY
        response = requests.get(baseURL)
        txnCount = response.json()['result']
        try:
            walletDF.loc[index, 'Txn Count'] = len(txnCount)
        
        except:
            pass

walletDF


,Unnamed: 0,WID,Label,Eth Balance,Txn Count,Exchange,ERC 20 Txn,ERC 20 Txn In,ERC 20 Txn Out,ERC721 Txn Out,ERC721 Txn In,ERC721 Txn
0,0,0x4dc98c79a52968a6c20ce9a7a08d5e8d1c2d5605,NaN,0.00477614 Ether,522.0,Y,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0xbb3fd383d1c5540e52ef0a7bcb9433375793aeaf,NaN,5.25604866 Ether,10000.0,Y,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0x1ccbdff6336b1027995a27a77b41fa87eb6608a3,NaN,0.06097497 Ether,7.0,Y,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0x05f51aab068caa6ab7eeb672f88c180f67f17ec7,ABCC,0 Ether,10000.0,Y,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0x2ddd202174a72514ed522e77972b461b03155525,Alcumex Exchange,0 Ether,709.0,Y,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
839,839,0xfba17aca0797f72bd8152c5e519100c0c4caf0ad,NaN,0.031192,967.0,N,NaN,NaN,NaN,NaN,NaN,NaN
840,840,0xfbbe05954c6b138999548171c272a1b109d89471,NaN,0.001795,255.0,N,NaN,NaN,NaN,NaN,NaN,NaN
841,841,0xfefa8d5093ffd962a094fdba722a2c30d1c7a822,NaN,0.035719,32.0,N,NaN,NaN,NaN,NaN,NaN,NaN
842,842,0xff64a8933e05c9d585ab72db95d207ebee9be5a8,NaN,30106.172274,22.0,N,NaN,NaN,NaN,NaN,NaN,NaN


Getting the ERC20 Transaction data.

In [34]:
for index, row in walletDF.iterrows():
    if pd.isna(row['ERC 20 Txn']):
        txnIn = 0
        txnOut = 0
        wallet = row['WID']
        baseURL = "https://api.etherscan.io/api?module=account&action=tokentx&address=" + wallet + "&startblock=0&endblock=99999999&sort=asc&apikey=" + API_KEY
        response = requests.get(baseURL)
        try:
            for txn in response.json()['result']:
                if txn['to'] == wallet:
                    txnIn += 1
                if txn['from'] == wallet:
                    txnOut += 1
            
            walletDF.loc[index, 'ERC 20 Txn In'] = txnIn
            walletDF.loc[index, 'ERC 20 Txn Out'] = txnOut
            walletDF.loc[index, 'ERC 20 Txn'] = txnIn + txnOut
        except:
            pass
walletDF

,Unnamed: 0,WID,Label,Eth Balance,Txn Count,Exchange,ERC 20 Txn,ERC 20 Txn In,ERC 20 Txn Out,ERC721 Txn Out,ERC721 Txn In,ERC721 Txn
0,0,0x4dc98c79a52968a6c20ce9a7a08d5e8d1c2d5605,NaN,0.00477614 Ether,522.0,Y,636.0,242.0,394.0,NaN,NaN,NaN
1,1,0xbb3fd383d1c5540e52ef0a7bcb9433375793aeaf,NaN,5.25604866 Ether,10000.0,Y,10000.0,7902.0,2098.0,NaN,NaN,NaN
2,2,0x1ccbdff6336b1027995a27a77b41fa87eb6608a3,NaN,0.06097497 Ether,7.0,Y,3.0,1.0,2.0,NaN,NaN,NaN
3,3,0x05f51aab068caa6ab7eeb672f88c180f67f17ec7,ABCC,0 Ether,10000.0,Y,10001.0,4287.0,5714.0,NaN,NaN,NaN
4,4,0x2ddd202174a72514ed522e77972b461b03155525,Alcumex Exchange,0 Ether,709.0,Y,626.0,35.0,591.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
839,839,0xfba17aca0797f72bd8152c5e519100c0c4caf0ad,NaN,0.031192,967.0,N,446.0,227.0,219.0,NaN,NaN,NaN
840,840,0xfbbe05954c6b138999548171c272a1b109d89471,NaN,0.001795,255.0,N,24.0,15.0,9.0,NaN,NaN,NaN
841,841,0xfefa8d5093ffd962a094fdba722a2c30d1c7a822,NaN,0.035719,32.0,N,0.0,0.0,0.0,NaN,NaN,NaN
842,842,0xff64a8933e05c9d585ab72db95d207ebee9be5a8,NaN,30106.172274,22.0,N,104.0,104.0,0.0,NaN,NaN,NaN


Getting the ERC721 Transaction data.

In [35]:
for index, row in walletDF.iterrows():
    
    if pd.isna(row['ERC721 Txn']):
        txnout = 0
        txnin = 0
        wallet = row['WID']
        baseURL = "https://api.etherscan.io/api?module=account&action=tokennfttx&address="+ wallet + "&startblock=0&endblock=99999999&sort=asc&apikey=" + API_KEY
        response = requests.get(baseURL)
        try: 
            for txn in response.json()['result']:
                if txn['from'] == wallet:
                    txnout += 1
                if txn['to'] == wallet:
                    txnin += 1
            walletDF.loc[index, 'ERC721 Txn Out'] = txnout
            walletDF.loc[index, 'ERC721 Txn In'] = txnin
            walletDF.loc[index, 'ERC721 Txn'] = txnout + txnin
        
        except:
            pass
walletDF        

,Unnamed: 0,WID,Label,Eth Balance,Txn Count,Exchange,ERC 20 Txn,ERC 20 Txn In,ERC 20 Txn Out,ERC721 Txn Out,ERC721 Txn In,ERC721 Txn
0,0,0x4dc98c79a52968a6c20ce9a7a08d5e8d1c2d5605,NaN,0.00477614 Ether,522.0,Y,636.0,242.0,394.0,0.0,0.0,0.0
1,1,0xbb3fd383d1c5540e52ef0a7bcb9433375793aeaf,NaN,5.25604866 Ether,10000.0,Y,10000.0,7902.0,2098.0,0.0,0.0,0.0
2,2,0x1ccbdff6336b1027995a27a77b41fa87eb6608a3,NaN,0.06097497 Ether,7.0,Y,3.0,1.0,2.0,0.0,0.0,0.0
3,3,0x05f51aab068caa6ab7eeb672f88c180f67f17ec7,ABCC,0 Ether,10000.0,Y,10001.0,4287.0,5714.0,0.0,0.0,0.0
4,4,0x2ddd202174a72514ed522e77972b461b03155525,Alcumex Exchange,0 Ether,709.0,Y,626.0,35.0,591.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
839,839,0xfba17aca0797f72bd8152c5e519100c0c4caf0ad,NaN,0.031192,967.0,N,446.0,227.0,219.0,NaN,NaN,NaN
840,840,0xfbbe05954c6b138999548171c272a1b109d89471,NaN,0.001795,255.0,N,24.0,15.0,9.0,NaN,NaN,NaN
841,841,0xfefa8d5093ffd962a094fdba722a2c30d1c7a822,NaN,0.035719,32.0,N,0.0,0.0,0.0,0.0,4.0,4.0
842,842,0xff64a8933e05c9d585ab72db95d207ebee9be5a8,NaN,30106.172274,22.0,N,104.0,104.0,0.0,0.0,0.0,0.0


In [ ]:
walletDF.to_csv('data.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=696215f1-387b-461d-aa4c-0ccf546cb4fd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>